In [1]:
import os 
import glob
import json
import numpy as np
import pandas as pd
import SimpleITK as sitk
import psycopg2
import matplotlib.pyplot as plt
from numpy.linalg import inv
import requests
import shutil
import re

import sys
sys.path.append(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration')
from ccf_slice_registration_functions import *

sys.path.append(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\lims_queries\cell_soma_structure_and_coords')
from soma_and_fiducial_pins import *

%matplotlib inline


Inputs

In [2]:
pins_path = get_soma_and_fiducial_pins()
pins = pd.read_csv(pins_path)

{'slide_specimen_id': 1069147202, 'specimen_name': 'Esr2-IRES2-Cre;Ai14-557045.04.09a'}
{'slide_specimen_id': 1069147202, 'specimen_name': 'Esr2-IRES2-Cre;Ai14-557045.04.09b'}
{'slide_specimen_id': 1069147202, 'specimen_name': 'Esr2-IRES2-Cre;Ai14-557045.04.09c'}
{'slide_specimen_id': 1102538034, 'specimen_name': 'Sst-IRES-Cre;Ai14-577302.07.04a', 'x': 4149.999999999999, 'y': 6388.183016393443, 'z': 8525.720459016393, 'structure_id': 0}
{'slide_specimen_id': 1068927614, 'specimen_name': 'Esr2-IRES2-Cre;Ai14-557044.05.09.03.01'}
{'slide_specimen_id': 1208614358, 'specimen_name': 'Curve'}
{'slide_specimen_id': 1262448162, 'specimen_name': 'Sst-IRES-Cre;Ai14-670419.09.03.02'}
{'slide_specimen_id': 1266114320, 'specimen_name': 'Gad2-IRES-Cre;Ai14-670842.06.02.01'}
{'slide_specimen_id': 1078776834, 'specimen_name': ' Htr3a-Cre_NO152;Ai14-563514.10.09.03'}


In [3]:
pins['specimen_name'] = pins['specimen_name'].str.strip() #strip any erroneous white space from the start and end fo specimen names

In [4]:
#pins contains fiducail pins (specimen name ends in a letter) and soma pins (specimen name ends in a number)
#break up pins into fiducial vs soma pin dataframes
fiducials_dict = {}
somas_dict = {}
for i, p in pins.iterrows():
    last_pin_char = p['specimen_name'][-1]
    if last_pin_char.isalpha(): fiducials_dict[i] = p #the last char of the pin name is a letter, so this is a fiducial (not a soma) pin 
    else: somas_dict[i] = p #the last char of the pin name is a number, so this is a soma (not a fiducial) pin
        

In [5]:
somas = pd.DataFrame.from_dict(somas_dict, orient='index').reset_index().drop(['index'], axis=1)
somas

,slide_specimen_id,specimen_name,x,y,z,structure_id,structure_acronym
0,1254192732,C57BL6J-667412.13.10.01,6725.000000,3807.984131,6180.958496,362,MD
1,1254192732,C57BL6J-667412.13.10.02,6725.000000,3685.511719,6276.875000,362,MD
2,1254192732,C57BL6J-667412.13.10.03,6725.000000,3607.036133,6302.043457,362,MD
3,1189282403,C57BL6J-634112.05.01.01,8846.905344,1215.824000,2322.970703,821,VISp2/3
4,1189282403,C57BL6J-634112.05.01.02,8698.307617,1624.090698,2576.191895,778,VISp5
...,...,...,...,...,...,...,...
2925,1170367254,C57BL6J-622939.07.04.02,4331.213867,4061.798584,6512.082520,672,CP
2926,1164113385,C57BL6J-618127.06.03.01,4275.000000,6365.933594,4770.908691,56,ACB
2927,1164113385,C57BL6J-618127.06.03.02,4275.000000,3978.592529,4848.456055,672,CP
2928,1164113385,C57BL6J-618127.06.03.03,4275.000000,4124.779297,3536.724121,672,CP


In [6]:
fiducials = pd.DataFrame.from_dict(fiducials_dict, orient='index').reset_index().drop(['index'], axis=1)
fiducials

,slide_specimen_id,specimen_name,x,y,z,structure_id,structure_acronym
0,1092005159,Glt25d2-Cre_NF107;Ai14-570372.07.09c,6975.000000,5882.571390,8955.783924,303,BLAa
1,1092005159,Glt25d2-Cre_NF107;Ai14-570372.07.09b,6975.000000,3045.212654,8379.161362,603,fi
2,1092005159,Glt25d2-Cre_NF107;Ai14-570372.07.09a,6975.000000,5294.686747,6272.187891,690,mtt
3,1110990523,Sst-IRES-Cre;Ai14-581875.06.04a,4450.000000,6401.548780,8550.109756,665,lot
4,1110990523,Sst-IRES-Cre;Ai14-581875.06.04b,4450.000000,5408.768293,6864.256098,900,aco
...,...,...,...,...,...,...,...
401,1170367254,C57BL6J-622939.07.04b,4532.975098,6367.930664,8660.227539,665,lot
402,1170367254,C57BL6J-622939.07.04c,4448.302525,5400.128877,6849.207031,900,aco
403,1164113385,C57BL6J-618127.06.03a,4275.000000,3468.452607,4859.073959,98,SEZ
404,1164113385,C57BL6J-618127.06.03b,4275.000000,6361.486503,2930.384694,665,lot


In [7]:
slices = fiducials.slide_specimen_id.unique() #get the slices that have fiducials
len(slices)

132

In [8]:
# #run julia's cells that failed the first time
# missing_cells = pd.read_csv(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\datasets\mouse_subcortical_msn_d1\10132023_ccf_reg\MSN_D1_Dataset_Sep_2023_failedCCFReg.csv')
# missing_cells



In [9]:
# missing_cell_list = missing_cells['Specimen Name'].values
# somas = somas[somas['specimen_name'].isin(missing_cell_list)]
# somas 


In [10]:
# slices = somas.slide_specimen_id.unique()
# slices

In [11]:
# slices = [1113439943]

In [12]:
# fiducials[fiducials['slide_specimen_id'].isin(slices)].sort_values(by='slide_specimen_id')


In [13]:
# somas[somas['slide_specimen_id'].isin(slices)].sort_values(by='slide_specimen_id')


In [14]:
out = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\ccf_reg_output_20231031' #folder to save slide transforms

In [15]:
# # cells = pd.read_csv(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\fiducals.csv', encoding='windows-1252') #slides with fiducials
# cells = pd.read_csv(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\ccf_reg_input\cells_with_fiducials_20230825.csv') #slides with fiducials
# # # cells = cells[cells['Fiducials drawn'] == 'complete + saved'] #get just cells with fiducials
# # slices = cells.slice_id.unique() #get the slices that have fiducials

# out = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\ccf_reg_output' #folder to save slide transforms

In [16]:
# # cells = pd.read_csv(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\fiducals.csv', encoding='windows-1252') #slides with fiducials
# # cells = pd.read_csv(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\ccf_reg_input\cells_with_fiducials_20230825.csv') #slides with fiducials
# cells = pd.read_csv(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\datasets\mouse_subcortical_msn_d1\09122023_ccf_reg\MSN_D1_Dataset_Sep_2023_formatted.csv')
# # # cells = cells[cells['Fiducials drawn'] == 'complete + saved'] #get just cells with fiducials
# slices = cells.slice_id.unique() #get the slices that have fiducials

# out = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\ccf_reg_output_msn_d1s' #folder to save slide transforms

Load CCF

In [17]:
# read in CCF (with fixed headers)
image_directory =  r'\\allen\programs\celltypes\workgroups\mousecelltypes\_UPENN_fMOST\mouse_ccf_fixed_headers_um\average_template'
ccf_file = os.path.join(image_directory, "average_template_10.nii.gz" )
ccf = sitk.ReadImage( ccf_file )

Transform slices to CCF

In [18]:
#loop through all slices, registering them to the ccf 
slices_with_issues = {}
for specimen_id in slices:
    specimen_id = int(specimen_id)
    # specimen_id = 1108154695 #a good ex

    #get slice name 
    specimen_name = get_name_by_id(specimen_id)
    print(specimen_name)

    try:

        #make folder to store transform for this slice 
        working_directory = os.path.join(out, specimen_name)
        if not os.path.exists(working_directory):
            os.mkdir(working_directory)

        # read in pinning tool output - virtual slice definition and cell ccf locations
        pinning_info = get_pinning(specimen_name)[4]

        # virtual_slice_to_ccf_transform: transform a 3D point (in micron) in the virtual slice to ccf (in micron)
        # ccf_to_virtual_slice_transform: transform a 3D point in ccf (in micron) to virtual slice (in micron)
        # (note: each point can have its own orientation - we are taking the first one only #TODO is this an issue? (swb))
        virtual_slice_to_ccf_transform = initialize_transform( pinning_info['markups'][0]['orientation'] )
        ccf_to_virtual_slice_transform = virtual_slice_to_ccf_transform.GetInverse()

        # write out the virtual_slice_to_ccf_transform
        file = os.path.join(working_directory, 'virtual_slice_to_ccf_transform.txt')
        sitk.WriteTransform( virtual_slice_to_ccf_transform, file )


        # generate a virtual slice from the transforms
        #   virtual_slice_3d is a 3D volume with a single z slice
        #   virtual_slice    is a 2D volume created by extracting out the single slice
        virtual_slice_size = [1250,1250,1] # 3D volume with one slice
        virtual_slice_spacing = ccf.GetSpacing()
        virtual_slice_3d = sitk.Resample(ccf, 
                                        virtual_slice_size, 
                                        virtual_slice_to_ccf_transform, 
                                        sitk.sitkLinear,
                                        [0,0,0], 
                                        virtual_slice_spacing, 
                                        [1,0,0,0,1,0,0,0,1], 
                                        0.0, 
                                        ccf.GetPixelID())#, False )


        virtual_slice = virtual_slice_3d[:,:,0] # 2D image version

        # read in the 2D overview image and associate metadata
        img_info = [list(get_20x_info(specimen_name))]

        odf = pd.DataFrame(img_info, columns = ['specimen_name', 'sub_image_id', 'width', 'height', 'resolution', 'treatment_id'])
        overview_info = odf.loc[0]
        sub_image = odf['sub_image_id'].values[0]
        get_20x_img(sub_image, specimen_name, working_directory)   
        file = os.path.join(working_directory, '{}_overview.jpg'.format(specimen_name))
        overview = sitk.ReadImage( file )
        overview_spacing = [overview_info['resolution'],overview_info['resolution']]
        overview.SetSpacing(overview_spacing)

        # downsample the 2D overview image
        downsampled_overview = sitk.BinShrink( sitk.VectorIndexSelectionCast(overview,0), [25,25])

        # write the downsampled overview to file
        file = os.path.join(working_directory,'downsampled_overview.nii.gz')
        sitk.WriteImage( downsampled_overview, file, True )

        # Read in drawn soma polygons to create matching landmarks set
        df = get_soma_polygons(specimen_id)

        # For each cell
        #  - compute cell soma from polyline in pixels
        #  - convert cell soma location to microns
        #  - join with cell soma location in CCF
        #  - compute cell soma location in virtual slice
        df['center_pixel'] = [compute_center_from_polyline(p) for p in df['poly_coords']]
        df['center_micron'] = [np.multiply(p,overview_spacing) for p in df['center_pixel']]

        jdict = {}
        for m in pinning_info['markups'] :
            jdict[m['name'].strip()] = m['markup']['controlPoints'][0]['position']
            
        df['ccf_coordinate'] = [jdict[p] for p in df['specimen_name']]
        df['virtual_slice_coordinate'] = [ ccf_to_virtual_slice_transform.TransformPoint(p)[:2] for p in df['ccf_coordinate'] ]
        #save df
        df.to_csv(os.path.join(working_directory, 'alignment_output.csv'), index=False)


        # for c in jdict.keys():
        #     print('|{}|'.format(c))

        lndmrks = get_landmark_ids(specimen_id)
        ldf = pd.DataFrame()
        for c in lndmrks:
            # print(c)
            this_lndmrk = get_landmark_location(c[0])
            ldf = pd.concat([ldf, this_lndmrk])
            
        ldf['center_pixel'] = [compute_center_from_polyline(p) for p in ldf['poly_coords']]
        ldf['overview_coordinate'] = [np.multiply(p,overview_spacing) for p in ldf['center_pixel']]

        ldf['ccf_coordinate'] = [jdict[p] for p in ldf['specimen_name']]
        ldf['virtual_slice_coordinate'] = [ ccf_to_virtual_slice_transform.TransformPoint(p)[:2] for p in ldf['ccf_coordinate'] ]

        if len(ldf) < 3:
            print('\tnot enough fiducials')
            # slices_with_issues[specimen_name] = 'not enough fiducials'
            continue

        # determine if slice was flipped, if so fix virtual to ccf transform 
        if slice_flipped(ldf):
            transform = virtual_slice_to_ccf_transform.GetParameters()
            transform = list(transform)
            idx = -1
            if pinning_info['referenceView'].lower() == 'sagittal': idx = 2 #flip coronal axis in PIL to LPS transform 
            if pinning_info['referenceView'].lower() == 'coronal': idx = 5 #flip saggital axis in RIA to LPS transform 
            if idx > -1: transform[idx] = transform[idx]*-1 #flip axis
            #TODO do I need to update one of the last three numbers in transform? 
            transform = tuple(transform)
            virtual_slice_to_ccf_transform.SetParameters(transform)

            # write out the virtual_slice_to_ccf_transform
            file = os.path.join(working_directory, 'virtual_slice_to_ccf_transform.txt')
            sitk.WriteTransform( virtual_slice_to_ccf_transform, file )

        # Concatentate landmarks (cell soma + additional) for registration
        # virtual slice (fixed) landmarks
        fixed_landmarks = [tuple(p) for p in df['virtual_slice_coordinate']]
        fixed_landmarks.extend([tuple(p) for p in ldf['virtual_slice_coordinate']] )

        # overview (moving) landmarks
        moving_landmarks = [tuple(p) for p in df['center_micron']]
        moving_landmarks.extend([tuple(p) for p in ldf['overview_coordinate']] )

        # virtual_slice_to_overview_transform: transform a 2D point (in microns) in the virtual slice to overview image (in microns)
        # overview_to_virtual_slice_transform: transform a 3D point in overview image (in microns) to virtual slice (in microns)
        virtual_slice_to_overview_transform = \
            sitk.LandmarkBasedTransformInitializer( sitk.AffineTransform(2), flatten(fixed_landmarks), flatten(moving_landmarks) )
        overview_to_virtual_slice_transform = virtual_slice_to_overview_transform.GetInverse()

        # write out the overview_to_virtual_slice_transform
        file = os.path.join(working_directory,'overview_to_virtual_slice_transform.txt')
        sitk.WriteTransform( overview_to_virtual_slice_transform, file )

        # for m in pinning_info['markups']:
        #     print(m['name'], m['markup']['type'])

        # generate resampled overview image
        resampled_overview = \
            sitk.Resample(downsampled_overview, virtual_slice, virtual_slice_to_overview_transform, \
                        sitk.sitkLinear, 0, downsampled_overview.GetPixelID())

        # write the downsampled overview to file
        file = os.path.join(working_directory,'resampled_overview.nii.gz')
        sitk.WriteImage( resampled_overview, file, True ) 


        fig, axes = plt.subplots(nrows=1,ncols=3,figsize=(15,5)) 
        visualize_landmarks(downsampled_overview, axes[0], moving_landmarks)
        axes[0].set_title('20x with fiducials')
        visualize_landmarks( virtual_slice, axes[1], fixed_landmarks)
        axes[1].set_title('virtual slice with fiducials')
        visualize_landmarks( resampled_overview, axes[2], fixed_landmarks)
        axes[2].set_title('resampled 20x')
        plt.savefig(os.path.join(working_directory, 'transformation_overview.jpg'))
        # plt.show()
        plt.clf()


    except: 
        print('\tissue with this slice {}'.format(specimen_id))
        slices_with_issues[specimen_name] = 'issue with this slice'
        continue


Glt25d2-Cre_NF107;Ai14-570372.07.09
spec: 1092058759
Sst-IRES-Cre;Ai14-581875.06.04
spec: 1111067076
Sst-IRES-Cre;Ai14-594119.11.02
spec: 1128233719
spec: 1128216050
spec: 1128211556
Vip-IRES-Cre;Ai14-579327.08.04
spec: 1105105529
spec: 1105112071
Vip-IRES-Cre;Ai14-585633.07.04
spec: 1115415523
Chat-IRES-Cre-neo;Ai14-600091.05.03
spec: 1136795168
spec: 1136784488
spec: 1136762178
spec: 1136724766
spec: 1136743496
Vip-IRES-Cre;Ai14-437116.02.01
spec: 811699569
spec: 810622500
	issue with this slice 810334213
C57BL6J-588656.06.04
spec: 1123293238
Chat-IRES-Cre-neo;Ai14-564233.06.09
spec: 1080362151
spec: 1080371567
Sst-IRES-Cre;Ai14-597074.09.02
spec: 1133681776
spec: 1133675234
Chat-IRES-Cre-neo;Ai14-573899.03.10
spec: 1097177576
spec: 1097187808
spec: 1097172863
Chat-IRES-Cre-neo;Ai14-576497.03.03
spec: 1099651737
spec: 1099667477
spec: 1099661552
Sst-IRES-Cre;Ai14-552310.11.09
spec: 1061496506
spec: 1061522940
spec: 1061508066
Chat-IRES-Cre-neo;Ai14-576499.01.04
spec: 1100133034
spec:

C:\Users\sarah.wallingbell\AppData\Local\Temp\ipykernel_16452\2978495204.py:159: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots(nrows=1,ncols=3,figsize=(15,5))


Sst-IRES-Cre;Ai14-554002.07.06
spec: 1064669498
Chat-IRES-Cre-neo;Ai14-600095.06.03
spec: 1137881239
spec: 1137872017
spec: 1137831952
Chat-IRES-Cre-neo;Ai14-600095.06.04
spec: 1137881770
spec: 1137869300
spec: 1137857452
spec: 1137849166
spec: 1137839115
spec: 1137831453
Vip-IRES-Cre;Ai14-597353.09.01
spec: 1132923582
spec: 1132931432
spec: 1132906901
	issue with this slice 1132314559
Rorb-IRES2-Cre-neo;Ai14-571899.05.10
spec: 1093670668
spec: 1093665779
Slc17a6-IRES-Cre;Ai14-578133.07.03
spec: 1103819612
spec: 1103808369
spec: 1103814311
Slc17a8-iCre;Slc32a1-IRES2-FlpO;Ai65-560685.08.06
spec: 1074972057
Sst-IRES-Cre;Ai14-601711.05.03
spec: 1139884869
spec: 1139874932
Slc32a1-IRES-Cre;Ai14-562359.09.09
spec: 1078105907
spec: 1078119654
Sst-IRES-Cre;Ai14-579950.09.03
spec: 1107412447
spec: 1107401239
spec: 1107392867
spec: 1107405539
spec: 1107398612
Sst-IRES-Cre;Ai14-572054.06.10
spec: 1093219269
spec: 1093227590
C57BL6J-622939.08.04
spec: 1171007170
spec: 1171002224
spec: 1170995444


<Figure size 1500x500 with 0 Axes>

<Figure size 1500x500 with 0 Axes>

...

<Figure size 1500x500 with 0 Axes>

<Figure size 1500x500 with 0 Axes>

<Figure size 1500x500 with 0 Axes>

<Figure size 1500x500 with 0 Axes>

<Figure size 1500x500 with 0 Axes>

In [19]:

slices_with_issues_df = pd.DataFrame.from_dict(slices_with_issues.items())
slices_with_issues_df.to_csv('slices_with_issues_20231031.csv', index=False)


Transform cell morphologies to CCF

In [20]:
# == To transform morphology file into CCF space
# 1) download the morphology from LIMS
# 2) if needed convert coordinates to micron - using the resolution of the 63x image
# 3) if needed correct for shrinkage - this should only affect the z coordinate! -swb add this for autotrace only cells
# 4) translate the (x,y) coorinates of the morphology such that the soma node is in the corresponding position in the overview image
# 5) transform the morphology to match the virtual slice
# ---- apply "overview_to_virtual_slice_transform" to the (x,y) coordinates of the morphology
# 6) transform from virtual slice to CCF space
# ---- apply "virtual_slice_to_ccf_transform" to the (x,y,z) coordinates of the morphology
# 7) transform back to PIR (CCF coords) from LPS (ITK coords) -swb
# 8) shift cell so soma is in pinned location in ccf (from json) -swb

Inputs

In [21]:
# cell_soma_path = get_cell_soma_coordinates()
# cell_soma_info = pd.read_csv(cell_soma_path)



# cell_soma_info = pd.read_csv(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\lims_queries\cell_soma_structure_and_coords\cell_soma_information_20230621.csv')


In [22]:
# cells = pd.read_csv(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\fiducals.csv', encoding='windows-1252') #slides with fiducials


In [23]:
# somas = somas.merge(cells, how='inner', left_on='specimen_name', right_on='Specimen Name')


In [24]:
len(somas)

2930

In [25]:
somas

,slide_specimen_id,specimen_name,x,y,z,structure_id,structure_acronym
0,1254192732,C57BL6J-667412.13.10.01,6725.000000,3807.984131,6180.958496,362,MD
1,1254192732,C57BL6J-667412.13.10.02,6725.000000,3685.511719,6276.875000,362,MD
2,1254192732,C57BL6J-667412.13.10.03,6725.000000,3607.036133,6302.043457,362,MD
3,1189282403,C57BL6J-634112.05.01.01,8846.905344,1215.824000,2322.970703,821,VISp2/3
4,1189282403,C57BL6J-634112.05.01.02,8698.307617,1624.090698,2576.191895,778,VISp5
...,...,...,...,...,...,...,...
2925,1170367254,C57BL6J-622939.07.04.02,4331.213867,4061.798584,6512.082520,672,CP
2926,1164113385,C57BL6J-618127.06.03.01,4275.000000,6365.933594,4770.908691,56,ACB
2927,1164113385,C57BL6J-618127.06.03.02,4275.000000,3978.592529,4848.456055,672,CP
2928,1164113385,C57BL6J-618127.06.03.03,4275.000000,4124.779297,3536.724121,672,CP


In [26]:
# somas = somas.query("specimen_name == 'Chat-IRES-Cre-neo;Ai14-585474.07.04.02'")
# somas

In [27]:
# name = 'C57BL6J-631344.08.04.01'
# cells = cells[cells['Specimen Name'] == name]
# cells

In [28]:
registered_cells = []
cells_with_issues = {}

for idx, cell in somas.iterrows():
    sp_name = cell['specimen_name']
    sl_name = sp_name.rsplit('.', 1)[0]

    print(sp_name)

    try:
        sp_id = get_id_by_name(sp_name)

        #check if there's a transform for this slice 
        slice_path = os.path.join(out, sl_name)
        if not os.path.isfile(os.path.join(slice_path, 'overview_to_virtual_slice_transform.txt')): continue #go on to next cell 
        if not os.path.isfile(os.path.join(slice_path, 'virtual_slice_to_ccf_transform.txt')): continue #go on to next cell 
        if not os.path.isfile(os.path.join(slice_path, 'alignment_output.csv')): continue #go on to next cell 


        #get soma loc in 20x 
        alignment_output = pd.read_csv(os.path.join(slice_path, 'alignment_output.csv'))
        this_cell_alignment = alignment_output[alignment_output.specimen_name == sp_name]
        if len(this_cell_alignment) != 1: 
            cells_with_issues[sp_name] = 'No soma pin'
            print('\t no soma pin')
            continue
        lims_soma = this_cell_alignment['center_micron'].values
        lims_soma = lims_soma[0][1:-2].split(' ')
        lims_soma = [float(i) for i in lims_soma if len(i) > 0]


        #make folder to save registered swcs
        swc_path = os.path.join(slice_path, 'SWC')
        if not os.path.exists(swc_path): os.mkdir(swc_path)


        #load affine transforms
        overview_to_virtual_slice_transform = sitk.ReadTransform(os.path.join(slice_path, 'overview_to_virtual_slice_transform.txt'))
        virtual_slice_to_ccf_transform = sitk.ReadTransform(os.path.join(slice_path, 'virtual_slice_to_ccf_transform.txt'))


        #register lims gold standard swc 
        try: 
            print('\tManual SWC:')
            swc_name = '{}'.format(sp_id)
            lims_path = list(get_swc_from_lims(str(sp_id)))[1]
            lims_path = edit_path(lims_path)
            morph = to_dict(lims_path)
            register_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                        overview_to_virtual_slice_transform, virtual_slice_to_ccf_transform)
            registered_cells = registered_cells + [sp_name+'_manual']
            print('\t\tMANUAL SWC REGISTERED!')
        except: print('\t\tno manual swc to register')

        #register autotrace raw swc
        try: 
            print('\tRaw autotrace SWC:')
            swc_name = '{}_autotrace_raw'.format(sp_id)
            raw_path = get_autotrace_raw_path(sp_id)
            # if os.path.isfile(raw_path):
            morph = to_dict(raw_path)
            morph = convert_pixel_to_um_dictnrn(morph, sp_id)
            register_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                    overview_to_virtual_slice_transform, virtual_slice_to_ccf_transform)
            registered_cells = registered_cells + [sp_name+'_raw']
            print('\t\tRAW AUTOTRACE SWC REGISTERED!')
            # else: print('\tno raw autotrace swc to register')
        except: print('\t\tno raw autotrace swc to register')

        #register autotrace pp14 swc
        try: 
            print('\tPost processed step 14 autotrace SWC:')
            swc_name = '{}_autotrace_pp14'.format(sp_id)
            pp14_path = get_autotrace_pp_path(sp_id, 14)
            # if os.path.isfile(pp14_path):
            morph = to_dict(pp14_path)
            morph = convert_pixel_to_um_dictnrn(morph, sp_id)
            register_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                    overview_to_virtual_slice_transform, virtual_slice_to_ccf_transform)
            registered_cells = registered_cells + [sp_name+'_pp14']
            print('\t\tPP14 AUTOTRACE SWC REGISTERED!')
            # else: print('\tno pp14 autotrace swc to register')
        except: print('\t\tno pp14 autotrace swc to register')

        
        #register autotrace pp22 swc
        try:
            print('\tPost processed step 22 autotrace SWC:')
            swc_name = '{}_autotrace_pp22'.format(sp_id)
            pp22_path = get_autotrace_pp_path(sp_id, 22)
            # if os.path.isfile(pp22_path):
            morph = to_dict(pp22_path)
            morph = convert_pixel_to_um_dictnrn(morph, sp_id)
            register_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                    overview_to_virtual_slice_transform, virtual_slice_to_ccf_transform)
            registered_cells = registered_cells + [sp_name+'_pp22']
            print('\t\tPP22 AUTOTRACE SWC REGISTERED!')
            # else: print('\tno pp22 autotrace swc to register')
        except: print('\t\tno pp22 autotrace swc to register')


    except: continue

C57BL6J-667412.13.10.01
C57BL6J-667412.13.10.02
C57BL6J-667412.13.10.03
C57BL6J-634112.05.01.01
C57BL6J-634112.05.01.02
C57BL6J-634112.05.01.03
Glt25d2-Cre_NF107;Ai14-570372.07.09.01
	Manual SWC:
		Starting step 1
		Starting step 3
		Starting step 4
		Starting step 5
		Starting step 6
		Starting step 7
		Starting step 8
		Registration complete!
		MANUAL SWC REGISTERED!
	Raw autotrace SWC:
		Starting step 1
		Starting step 3
		Starting step 4
		Starting step 5
		Starting step 6
		Starting step 7
		Starting step 8
		Registration complete!
		RAW AUTOTRACE SWC REGISTERED!
	Post processed step 14 autotrace SWC:
		Starting step 1
		Starting step 3
		Starting step 4
		Starting step 5
		Starting step 6
		Starting step 7
		Starting step 8
		Registration complete!
		PP14 AUTOTRACE SWC REGISTERED!
	Post processed step 22 autotrace SWC:
		Starting step 1
		Starting step 3
		Starting step 4
		Starting step 5
		Starting step 6
		Starting step 7
		Starting step 8
		Registration complete!
		PP22 AUTOT

In [29]:
cells_with_issues_df = pd.DataFrame.from_dict(list(cells_with_issues.items())) 
cells_with_issues_df.to_csv('cells_with_issues_20231031.csv', index=False)

In [30]:
registered_cells

['Glt25d2-Cre_NF107;Ai14-570372.07.09.01_manual',
 'Glt25d2-Cre_NF107;Ai14-570372.07.09.01_raw',
 'Glt25d2-Cre_NF107;Ai14-570372.07.09.01_pp14',
 'Glt25d2-Cre_NF107;Ai14-570372.07.09.01_pp22',
 'Sst-IRES-Cre;Ai14-581875.06.04.01_manual',
 'Sst-IRES-Cre;Ai14-581875.06.04.01_raw',
 'Sst-IRES-Cre;Ai14-581875.06.04.01_pp14',
 'Sst-IRES-Cre;Ai14-581875.06.04.01_pp22',
 'Sst-IRES-Cre;Ai14-594119.11.02.01_raw',
 'Sst-IRES-Cre;Ai14-594119.11.02.01_pp14',
 'Sst-IRES-Cre;Ai14-594119.11.02.01_pp22',
 'Sst-IRES-Cre;Ai14-594119.11.02.02_raw',
 'Sst-IRES-Cre;Ai14-594119.11.02.02_pp14',
 'Sst-IRES-Cre;Ai14-594119.11.02.02_pp22',
 'Sst-IRES-Cre;Ai14-594119.11.02.03_raw',
 'Sst-IRES-Cre;Ai14-594119.11.02.03_pp14',
 'Sst-IRES-Cre;Ai14-594119.11.02.03_pp22',
 'Vip-IRES-Cre;Ai14-579327.08.04.01_manual',
 'Vip-IRES-Cre;Ai14-579327.08.04.01_raw',
 'Vip-IRES-Cre;Ai14-579327.08.04.01_pp14',
 'Vip-IRES-Cre;Ai14-579327.08.04.01_pp22',
 'Vip-IRES-Cre;Ai14-585633.07.04.01_manual',
 'Vip-IRES-Cre;Ai14-585633.07.04

In [31]:
registered_cells_df = pd.DataFrame(registered_cells)
registered_cells_df

,0
0,Glt25d2-Cre_NF107;Ai14-570372.07.09.01_manual
1,Glt25d2-Cre_NF107;Ai14-570372.07.09.01_raw
2,Glt25d2-Cre_NF107;Ai14-570372.07.09.01_pp14
3,Glt25d2-Cre_NF107;Ai14-570372.07.09.01_pp22
4,Sst-IRES-Cre;Ai14-581875.06.04.01_manual
...,...
459,C57BL6J-622939.07.04.02_raw
460,C57BL6J-618127.06.03.01_manual
461,C57BL6J-618127.06.03.01_raw
462,C57BL6J-618127.06.03.04_manual


In [32]:
registered_cells_df.to_csv('registered_cells_20231031.csv', index=False)